In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!nvidia-smi

In [ ]:
!pip install  -U trl  git+https://github.com/huggingface/peft.git
!pip install  datasets 
!pip install bitsandbytes 
!pip install einops 
!pip install wandb 
!pip install transformers 
!pip install accelerate

In [ ]:
import os
import getpass

#os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("API_Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

In [ ]:
!huggingface-cli login --token $HUGGING_FACE_HUB_TOKEN

In [ ]:
from datasets import load_dataset

dataset_name = 'Ekkologico/phyton-clean-tiny-codes' 
dataset = load_dataset(dataset_name, split="train", use_auth_token=True)

In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer



model_name = "meta-llama/Llama-2-13b-chat-hf"
MAX_NEW_TOKENS = 128


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

device_map = "auto"
max_memory = {0: "13GiB", 1: "13GiB", "cpu": "28GiB"}

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map = device_map,
    max_memory = max_memory
)
model.config.use_cache = False

In [ ]:
model.hf_device_map

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"
eval_accumulation_steps=1

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    eval_accumulation_steps=eval_accumulation_steps,
    gradient_checkpointing=True
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="combined",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
text = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Create a Python function that takes in a string and a list of words and returns true if the string contains all the words in the list. ### Input: 'This is a test', ['test', 'this', 'is']"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_folder(
    folder_path="./outputs",
    path_in_repo = ".",
    repo_id="Ekkologico/Llama-2-13b-chat-python_code_instructions_nampdn-ai_tiny-codes",
    repo_type="model"
)